In [2]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization

NUM_EPOCHS = 70
INIT_LR = 5e-3
COLORS = 1
KERN = 3 

#create an object of the sequential class
model = Sequential()

#Layer 1- add 2D convolutional layer to process the 2D input images. We have 32 output channels.
#Next we have a 3x3 moving window (kernel_size) and declearing input shape is only reqired of the first layer.
model.add(Conv2D(32, (KERN, KERN), input_shape = (64, 64, COLORS)))


#add the pooling layer- here we don't need to declear weights or bias variables like in tflearn. Keras sorts that out for us.
model.add(MaxPooling2D(pool_size = (2, 2), strides= (2,2))) #strides= x and y directions(2,2)
#use rectified linear unit as activation function
model.add(Activation('relu'))
model.add(BatchNormalization())

# Layer 2
model.add(Conv2D(32, (KERN, KERN)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Conv2D(64, (KERN, KERN), activation='relu'))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

#convert the pooled images into a 1D features vector
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
#initilize output layer
model.add(Dense(6, activation = 'softmax')) #6 labels

opt = SGD(lr=INIT_LR, momentum=0.9)
#opt = 'adam'
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])


/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
batch_size = 16
train_datagen = ImageDataGenerator( #rescale = 1./255 
featurewise_center=False, # set input mean to 0 over the dataset
samplewise_center=False, # set each sample mean to 0
featurewise_std_normalization=False, # divide inputs by std of the dataset
samplewise_std_normalization=False, # divide each input by its std
#zca_whitening=False, # apply ZCA whitening
rotation_range=1, # randomly rotate images in the range (degrees, 0 to 180)
width_shift_range=0.1, # randomly shift images horizontally (fraction of total width)
height_shift_range=0.1, # randomly shift images vertically (fraction of total height)
horizontal_flip=False, # randomly flip images
vertical_flip=False) # randomly flip images


test_datagen = ImageDataGenerator( #rescale = 1./255 
#featurewise_center=False, # set input mean to 0 over the dataset
samplewise_center=False, # set each sample mean to 0
#featurewise_std_normalization=False, # divide inputs by std of the dataset
samplewise_std_normalization=False, # divide each input by its std
#zca_whitening=False, # apply ZCA whitening
#rotation_range=1, # randomly rotate images in the range (degrees, 0 to 180)
#width_shift_range=0.1, # randomly shift images horizontally (fraction of total width)
#height_shift_range=0.1, # randomly shift images vertically (fraction of total height)
#horizontal_flip=False, # randomly flip images
vertical_flip=False) # randomly flip images
#rescale = 1./255)

td = 'Marcel-train'
TRAIN_IMG_CNT = 4872
#td = 'more-train'
#TRAIN_IMG_CNT = 19488
VALIDATION_CNT = 659

training_set = train_datagen.flow_from_directory(td, 
                                                 target_size = (64, 64), 
                                                 batch_size = batch_size,
                                                 color_mode = 'grayscale',
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('Marcel-Test', 
                                            target_size = (64, 64), 
                                            batch_size = batch_size, 
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 4872 images belonging to 6 classes.
Found 659 images belonging to 6 classes.


In [4]:
def poly_decay(epoch):
	# initialize the maximum number of epochs, base learning rate,
	# and power of the polynomial
	maxEpochs = NUM_EPOCHS
	baseLR = INIT_LR
	power = 1.0
 
	# compute the new learning rate based on polynomial decay
	alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power
 
	# return the new learning rate
	return alpha

In [5]:
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping

early_stop_monitor = EarlyStopping(patience = 5)
callbacks = [
    LearningRateScheduler(poly_decay),
#            early_stop_monitor
    ]


In [6]:

model.fit_generator(training_set, steps_per_epoch = TRAIN_IMG_CNT / batch_size, epochs = NUM_EPOCHS,
                    callbacks=callbacks,
                    validation_data = test_set, validation_steps = VALIDATION_CNT / batch_size)
model.save_weights('1st_run.h5') 

Epoch 1/70
305/304 [==============================] - 19s 62ms/step - loss: 1.4734 - acc: 0.3996 - val_loss: 2.0697 - val_acc: 0.2428
Epoch 2/70
305/304 [==============================] - 21s 70ms/step - loss: 1.2602 - acc: 0.4730 - val_loss: 1.9668 - val_acc: 0.2398
Epoch 3/70
305/304 [==============================] - 25s 83ms/step - loss: 1.1465 - acc: 0.5244 - val_loss: 1.7229 - val_acc: 0.3156
Epoch 4/70
305/304 [==============================] - 22s 73ms/step - loss: 1.0848 - acc: 0.5520 - val_loss: 1.6823 - val_acc: 0.3323
Epoch 5/70
305/304 [==============================] - 21s 69ms/step - loss: 0.9702 - acc: 0.6072 - val_loss: 2.0560 - val_acc: 0.3520
Epoch 6/70
305/304 [==============================] - 22s 71ms/step - loss: 0.8749 - acc: 0.6572 - val_loss: 2.5272 - val_acc: 0.3869
Epoch 7/70
305/304 [==============================] - 21s 68ms/step - loss: 0.7462 - acc: 0.7123 - val_loss: 1.3697 - val_acc: 0.5630
Epoch 8/70
305/304 [==============================] - 20s 65ms

Epoch 62/70
305/304 [==============================] - 18s 58ms/step - loss: 0.0966 - acc: 0.9625 - val_loss: 1.5322 - val_acc: 0.6980
Epoch 63/70
305/304 [==============================] - 19s 63ms/step - loss: 0.0895 - acc: 0.9676 - val_loss: 1.5431 - val_acc: 0.6950
Epoch 64/70
305/304 [==============================] - 18s 59ms/step - loss: 0.0939 - acc: 0.9670 - val_loss: 1.5101 - val_acc: 0.6995
Epoch 65/70
305/304 [==============================] - 18s 60ms/step - loss: 0.0907 - acc: 0.9684 - val_loss: 1.5433 - val_acc: 0.6965
Epoch 66/70
305/304 [==============================] - 18s 60ms/step - loss: 0.0987 - acc: 0.9648 - val_loss: 1.5635 - val_acc: 0.6935
Epoch 67/70
305/304 [==============================] - 19s 61ms/step - loss: 0.0819 - acc: 0.9682 - val_loss: 1.5037 - val_acc: 0.7011
Epoch 68/70
305/304 [==============================] - 18s 60ms/step - loss: 0.0829 - acc: 0.9701 - val_loss: 1.5465 - val_acc: 0.6980
Epoch 69/70
305/304 [==============================] - 

In [8]:
import keras
print(keras.__version__)

2.1.5
